## Install Required Libraries for RAG Chatbot:

- Langchain Libraries and LLM (from GROQ to get public chatbot with free trial).
- Database.
- Embeddings.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

%pip install langchain langchain_community faiss-cpu sentence_transformers groq langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

## LOAD & SPLIT DOCUMENTS:

- Create functions to load documents and split texts, chunks of spliting is 512, and overlap 10% - 20% from chunks, choosen around 15% = 75.

In [15]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_split(doc):
    loader = TextLoader(doc)
    all_text = loader.load()

    # split text:
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120, length_function=len)
    chunks = splitter.split_text(all_text[0].page_content)

    print(f"number of chunks: {len(chunks)}")

    return chunks




**Call loader and split function**

In [16]:
chunks = load_split("/content/drive/MyDrive/concateneted_docs.txt")

number of chunks: 3882


## LOAD EMBEDDING, AND CREATE VECTOR DATABASE:

- The embedding model used is E5-base-v2 depending the benchmark results and the porpuse of RAG chatbot.
- Used FAISS storage to store vectors created from embedding.

In [10]:
import numpy as np
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer

In [11]:
MODEL_EMB = SentenceTransformer("intfloat/e5-base-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [17]:



def get_emb_and_idx(text, model_emb=MODEL_EMB):

    embeddings = model_emb.encode(text, show_progress_bar=True)
    emb_np = np.array(embeddings).astype('float32')

    dim = emb_np.shape[1]

    index = faiss.IndexFlatL2(dim)

    faiss.normalize_L2(emb_np)

    index.add(emb_np)

    # store the index and dataframe:
    faiss.write_index(index, "/content/drive/My Drive/faiss_index_800_chunks.bin")

    data = {'id' : range(len(text)), 'text' : text}

    df = pd.DataFrame(data)
    df.to_csv("/content/drive/My Drive/documents_800_chunks.csv", index=False)

    print(f"Embadding done and stored in index of FAISS with {len(text)} chunks.")





**Call embedding function to create & store vectors**

In [18]:
get_emb_and_idx(chunks)

Batches:   0%|          | 0/122 [00:00<?, ?it/s]

Embadding done and stored in index of FAISS with 3882 chunks.


## BUILD RAG CHAT MODEL:

- Use one of groq models as free trial.

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import os

In [ ]:
# Load FAISS index and DataFrame
print("Loading database...")
index = faiss.read_index("faiss_index.bin")
df = pd.read_csv("documents.csv")


prompt_tamplate = ChatPromptTemplate([("system", """You're AI assistant that help to learn Machine Learning-ML in specific and other related topic of it.
                                       Take question, and break it, then answer each part, answer in points.
                                       You will give contexts that help you to generate the answer, please follow them and let the answer clear and understandable for person how learn about it as first time."""),
                                       ("human", "Context: {context}, question {question}")])


llm = ChatGroq(model="qwen/qwen3-32b", temperature=0.4, groq_api_key=os.environ.get("GROQ_API_KEY"), max_tokens=512)

def rag_chat(question, k=4, index=index, df=df, tamplete=prompt_tamplate, llm=llm):

    # embedding question:
    query = MODEL_EMB.encode(question)
    query_arr = np.array(query).astype('float32')

    faiss.normalize_L2(query_arr)

    # search:
    destances, indices = index.search(query_arr, k)

    retrived_texts = [df.at[i, "text"] for i in indices[0]]

    # message and prompt:
    message = tamplete.format_messages(context=retrived_texts, question=question)

    # chat model:
    respons = llm.invoke(message)

    return respons.content



In [ ]:

while True:

    print("--- Welcome to ML Teaching ChotBot ---\n\n\n")

    human_ans = input("Ask your Question or (quit) to exist: ")

    if human_ans.lower() == "quit":
        print("I hope that I helped you, see you soon!")
        break


    try:
        ai_ans = rag_chat(human_ans)
        print('-'* 70)
        print(ai_ans)

    except Exception as e:
        print("Sorry, error happend.")

